In [4]:
import spotipy
import spotipy.util as util
import numpy as np
import pandas as pd
import os

In [5]:
absolute_path = "f:\\AI Projects\\Music Classification"
relative_path = "Data\\Spotify"
full_path = os.path.join(absolute_path, relative_path)

In [6]:
CLIENT_ID = "372d9bfe8dce48a1aac1806d4329629d"
CLIENT_SECRET = "1a6e5a340428469dba631f9ffbd4528f"

In [7]:
auth_manager = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
def get_playlist_tracks(playlist_id):
    playlist_data = sp.playlist_tracks(playlist_id)
    tracks = playlist_data['items']
    while playlist_data['next']:
        playlist_data = sp.next(playlist_data)
        tracks.extend(playlist_data['items'])
        
    return tracks

Get tracks from playlists

In [49]:
playlists = ["37i9dQZF1DWXRqgorJj26U",
"2MepDCzimtCz2ytAlLUdUi",
"1h0CEZCm6IbFTbxThn6Xcs",]
# "37i9dQZF1DX8qqIDAkKiQg",
# "7pWNG89bI9uwHQaUI3pINb",
# "37i9dQZF1DX12YIlzNkccL"] #["4rnleEAOdmFAbRcNCgZMpY","37i9dQZF1DWWEJlAGA9gs0","37i9dQZF1DXbITWG1ZJKYt","37i9dQZF1DX1spT6G94GFC","5oM9tyhspzK18TrPrlIZro","37i9dQZF1DX55yuR78Invt"]

Define all artists

In [50]:
from itertools import chain
genre_count = {}
broad_genres = ['rock','metal','pop','rap','contemporary','hip hop','r&b','classical','jazz','reggae','soul']


def get_broad_genres(genres):
    mapped_genres = []
    
    for g in genres:
        mapped_genres.append([g1 for g1 in broad_genres if g1 in g])

    mapped_genres = list(chain.from_iterable(mapped_genres))
    mapped_genres = mapped_genres = list(filter(None, mapped_genres))

    if mapped_genres: 
        mapped_genres = np.unique(mapped_genres)
        for g in mapped_genres:
            if g in genre_count:
                genre_count[g] = genre_count.get(g)  + 1
            else:
                genre_count[g] = 1
        return mapped_genres
    else:
        print("No broad genres found")
        return None


In [51]:
import warnings
warnings.filterwarnings('ignore')

def get_artists_from_playlist(playlist_url):
    columns = ["artist","uri","genres","broad_genres"]
    df = pd.DataFrame(columns=columns)
    total_artists = 0

    for playlist in playlist_url:
        playlist_data = sp.playlist_tracks(f"https://open.spotify.com/playlist/{playlist}")
        tracks = playlist_data['items'] 
        while playlist_data['next']:
            playlist_data = sp.next(playlist_data)
            tracks.extend(playlist_data['items'])
            
        for track in tracks:
            name = track["track"]["artists"][0]["name"]
            uri = track["track"]["artists"][0]["uri"]
            if not (df['artist'].str.contains(name, regex=False).any() & 
            df['uri'].str.contains(uri, regex=False).any()):
                genres = sp.artist(uri)["genres"]
                broad_genres = get_broad_genres(genres)
                new_row = {"artist" : name, "uri" : uri, "genres" : genres, "broad_genres" : broad_genres}
                df = df.append(new_row, ignore_index=True)
                total_artists += 1
            else:
                print(f"Skipping: {name} due to a duplicate")

    print(f"Total number of artists: {total_artists}")
    return df

metadata_artists = get_artists_from_playlist(playlists)
metadata_artists.head()


Skipping: AC/DC due to a duplicate
Skipping: Fleetwood Mac due to a duplicate
Skipping: Queen due to a duplicate
Skipping: Van Halen due to a duplicate
No broad genres found
Skipping: Bon Jovi due to a duplicate
Skipping: Pink Floyd due to a duplicate
Skipping: Van Halen due to a duplicate
Skipping: The Rolling Stones due to a duplicate
Skipping: Queen due to a duplicate
Skipping: Guns N' Roses due to a duplicate
Skipping: Led Zeppelin due to a duplicate
Skipping: ZZ Top due to a duplicate
Skipping: Bruce Springsteen due to a duplicate
Skipping: The Police due to a duplicate
Skipping: Creedence Clearwater Revival due to a duplicate
Skipping: AC/DC due to a duplicate
Skipping: David Bowie due to a duplicate
Skipping: Eagles due to a duplicate
Skipping: Red Hot Chili Peppers due to a duplicate
Skipping: Lynyrd Skynyrd due to a duplicate
Skipping: Nirvana due to a duplicate
Skipping: Fleetwood Mac due to a duplicate
Skipping: U2 due to a duplicate
Skipping: Foo Fighters due to a duplicate

In [7]:
artists = [
"36QJpDe2go2KgaRleHCDTp",
"6mdiAmATAx73kdxrNrnlao",
"2ye2Wgw4gimLv2eAKyk1NB",
"58lV9VcRSjABbAbfWS6skp",
"27T030eWyCQRmDyuvr1kxY",
"26bcq2nyj5GB7uRr558iQg",
"6olE6TJLqED3rqDCT0FyPh",
"0rvjqX7ttXeg3mTy8Xscbt",
"711MCceyCBcFnzjGY4Q7Un",
"3qm84nBOXUEQ2vnTfUTTFC",
"7Js6Lde8thlIHXggv2SCBz",
"6KImCVD70vtIoJWnq6nGn3",
"0Cp8WN4V8Tu4QJQwCN5Md4",
"6qqNVTkY8uBg9cP3Jd7DAH",
"06HL4z0CvFAxyc27GXpf02",
"4AK6F7OLvEQ5QYCBNiQWHq",
"4dpARuHxo51G3z768sgnrY",
"6eUKZXaKkcviH0Ku9w2n3V",
"08GQAI4eElDnROBrJRGE0X",
"2cnMpRsOVqtPMfq7YiFE6K",
"6ZLTlhejhndI4Rh53vYhrY",
"5M52tdBnJaKSvOpJGz8mfZ",
"3EhbVgyfGd7HkpsagwL9GS",
"0LcJLqbBmaGUft1e9Mm8HV",
"1LZEQNv7sE11VDY3SdxQeN",
"54R6Y0I7jGUCveDTtI21nb",
"1HY2Jd0NmPuamShAr6KMms",
"6jJ0s89eD6GaHleKKya26X",
"7hJcb9fa4alzcOq3EaNPoG",
"0du5cEVh5yTK9QJze8zA0C",
"1aSxMhuvixZ8h9dK9jIDwL",
"3PhoLpVuITZKcymswpck5b",
"6zFYqv1mOsgBRQbae3JJ9e",
"6M2wZ9GZgrQXHCFfjv46we",
"1Xyo4u8uXC1ZmMpatF05PJ",
"5pKCCKE2ajJHZ9KAiaK11H",
"7dGJo4pcD2V6oG8kP0tJRR",
"6vWDO969PvNqNYHIOW5v0m",
"6tbjWDEIzxoDsBA1FuhfPW",
"0PFtn5NtBbbUNbU9EAmIWF",
"1McMsnEElThX1knmY4oliG",
"4V8LLVI7PbaPR0K2TGSxFF",
"15UsOTVnJzReFVN1VCnxy4",
"6DPYiyq5kWVQS4RGwxzPC7",
"7B4hKK0S9QYnaoqa9OuwgX",
"4EnEZVjo3w1cwcQYePccay",
"4MCBfE4596Uoi2O4DtmEMz",
"1Bl6wpkWCQ4KVgnASpvzzA",
"3CQIn7N5CuRDP8wEI7FiDA",
"6XyY86QOPPrYVGvF9ch6wz",
"4gzpq5DPGxSnKTe4SA8HAU",
"2YZyLoL8N0Wb9xBt1NhZWg",
"0Y5tJX1MQlPlqiwlOH1tJY",
"5K4W6rqBFWDnAN6FQUkS6x",
"0hCNtLu0JehylgoiP8L4Gh",
"3TVXtAsR1Inumwj472S9r4",
"4O15NlyKLIASxsJ0PrXPfz",
"699OTQXzgjhIYAHMy9RyPD",
"246dkjvS1zLTtiykXe5h60",
"5f7VJjfbwm532GiveGC0ZK",
]

Get top tracks from artists

In [9]:
import warnings
warnings.filterwarnings('ignore')

columns = ["artist","album_name","track_name","track_id","released_date","explicit","genres","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms","time_signature"]
metadata_toptracks = pd.DataFrame(columns=columns)
total_tracks = 0

for artist_uri in artists:
    artist = sp.artist(artist_uri)["name"]
    results = sp.artist_top_tracks(artist_uri) 
    genres = sp.artist(artist_uri)["genres"]
    genres = update_genre_count(genres)
    for track in results["tracks"][:10]:
        album_name = track["album"]["name"]
        track_name = track["name"]
        if not (
            metadata_toptracks['artist'].str.contains(artist, regex=False).any() & 
            metadata_toptracks['album_name'].str.contains(album_name, regex=False).any() & 
            metadata_toptracks['track_name'].str.contains(track_name, regex=False).any()):
            # metadata_toptracks.duplicated(subset=['artist','album_name','track_name'], keep=False)
            track_id = track["id"]
            released_date = track["album"]["release_date"]
            explicit = track["explicit"]
            new_row = {"artist" : artist, "album_name" : album_name, "track_name" : track_name, "track_id" : track_id,"released_date" : released_date, "explicit" : explicit, "genres" : genres}
            audio_features = sp.audio_features(track_id)[0]
            for feature in columns[7:]:
                if audio_features is None:
                    print(f"Skipping: {track_name} due to no audio features")
                    break
                new_row[feature] = audio_features[feature]
            if audio_features is not None:
                metadata_toptracks = metadata_toptracks.append(new_row, ignore_index=True)
                total_tracks += 1
        else:
            print(f"Skipping: {track_name} due to a duplicate")
print(f"Total number of tracks processed: {total_tracks}")

KeyboardInterrupt: 

In [27]:
genre_count

{'rock': 25,
 'metal': 14,
 'pop': 23,
 'soul': 1,
 'hip hop': 14,
 'rap': 20,
 'contemporary': 2,
 'r&b': 2}

In [33]:
metadata_toptracks.shape

(588, 20)

In [34]:
metadata_toptracks.head()

,artist,album_name,track_name,track_id,released_date,explicit,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Led Zeppelin,Led Zeppelin IV (Deluxe Edition),Stairway to Heaven - Remaster,5CQ30WqJwcep0pYcV4AMNc,1971-11-08,False,[rock],0.338,0.340,9,-12.049,0,0.0339,0.580000,0.00320,0.1160,0.197,82.433,482830,4
1,Led Zeppelin,Led Zeppelin III (Remaster),Immigrant Song - Remaster,78lgmZwycJ3nzsdgmPPGNx,1970,False,[rock],0.564,0.932,11,-10.068,1,0.0554,0.013000,0.16900,0.3490,0.619,112.937,146250,4
2,Led Zeppelin,Led Zeppelin II (1994 Remaster),Whole Lotta Love - 1990 Remaster,0hCB0YR03f6AmQaHbwWDe8,1969-10-22,False,[rock],0.412,0.902,9,-11.600,1,0.4050,0.048400,0.13100,0.4050,0.422,89.740,333893,4
3,Led Zeppelin,Led Zeppelin IV (Deluxe Edition),Black Dog - Remaster,3qT4bUD1MaWpGrTwcvguhb,1971-11-08,False,[rock],0.437,0.864,4,-7.842,0,0.0904,0.396000,0.03140,0.2420,0.749,81.394,295387,4
4,Led Zeppelin,Led Zeppelin IV (Deluxe Edition),Rock and Roll - Remaster,4PRGxHpCpF2yoOHYKQIEwD,1971-11-08,False,[rock],0.317,0.887,9,-7.292,1,0.0375,0.000582,0.00258,0.0891,0.871,169.613,220561,4


In [35]:
metadata_toptracks.to_csv(os.path.join(full_path,"metadata_toptracks.csv"))